# Extreme Natural events: Cyclones

## Data Extraction from EONET NASA API

In [233]:
import pandas as pd
import numpy as np
import re
import math
import requests
import json
import matplotlib
from pandas import ExcelFile
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.express
import reverse_geocoder as rg
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


The natural events in the EONET's API are divided in two primarily groups according with their status:

* ```'status' = 'open'``` means the natural event has not ended.

* ```'status' = 'closed'``` means the natural event has finished. When this happen we will have as well an aditional field labeled as closed.

## 1. Data extraction and wrangling

### Extracting data from the EONET's API

In [3]:
#For extracting the category of each natural event

def extract_cat(lista):
    """
    Takes a list of dictionaries and return the value of the first element
    of the list for the key 'title'
    """
    return lista[0]["title"]


# Extracts the date of each natural event
def extract_date(lista):
    """
    Takes a list of dictionaries and return the value of the first element
    of the list for the key 'date'
    """
    return lista[0]["date"]

# Extracts the type of Geojson coordinates of each natural event

def extract_geom(lista):
    """
    Takes a list of dictionaries and removes the key 'date' for the first 
    dictionary
    """
    del lista[0]["date"]
    return lista[0]
    
base = "https://eonet.sci.gsfc.nasa.gov/api/v2.1/events?status=" 

def get_events(st):
    
    """
    Arguments: "open" or "closed"
    Returns a the data set of the events with status "st" 
    which happened since 1980-01-10 until teh current date.
    
    """
    url = base + f"{st}"
    result = requests.get(url)
    
    df_0 = pd.DataFrame(result.json()["events"], columns = ["id", "title", "categories", "description", "sources", "geometries", "closed"])
    df_0.categories = df_0.categories.apply(extract_cat) # Extract category from the original dictionary
    df_0["date"] = df_0.geometries.apply(extract_date)
    df_0["geometries"] = df_0.geometries.apply(extract_geom)
    df_0 = df_0.drop(columns = ["sources"])
    return df_0[['id', 'title', 'categories', 'description', 'date', 'closed','geometries']]


In [4]:
df_op = get_events("open")
df_cl = get_events("closed")

In [5]:
df_ev = (pd
        .concat([df_op, df_cl], ignore_index=True)
        .sort_values("date", ascending = False)
        .reset_index(drop=True))

df_ev.dtypes

id             object
title          object
categories     object
description    object
date           object
closed         object
geometries     object
dtype: object

In [6]:
df_ev.head()

,id,title,categories,description,date,closed,geometries
0,EONET_4617,"Wildfire - Molina, Chile",Wildfires,,2020-03-16T17:18:00Z,NaN,"{'type': 'Point', 'coordinates': [-71.28198, -..."
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13T00:00:00Z,NaN,"{'type': 'Point', 'coordinates': [52.4, -14.3]}"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13T00:00:00Z,NaN,"{'type': 'Point', 'coordinates': [153.8, -16.9]}"
3,EONET_4614,"Wildfire - Constitucion, Chile",Wildfires,,2020-03-11T15:37:00Z,NaN,"{'type': 'Point', 'coordinates': [-72.42013, -..."
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11T00:00:00Z,2020-03-16T00:00:00Z,"{'type': 'Point', 'coordinates': [116.9, -16.4]}"


In [12]:
df_ev.categories.value_counts()

Severe Storms           1825
Wildfires               1535
Volcanoes                864
Floods                   116
Sea and Lake Ice          60
Water Color               17
Dust and Haze             17
Earthquakes               15
Temperature Extremes      14
Drought                    7
Manmade                    5
Snow                       3
Landslides                 2
Name: categories, dtype: int64

In [8]:
df_ev["date"]= pd.to_datetime(df_ev['date'],infer_datetime_format = True).dt.date
df_ev["closed"]= pd.to_datetime(df_ev['closed'],infer_datetime_format = True).dt.date

In [9]:
df_ev["date"]= pd.to_datetime(df_ev['date'],infer_datetime_format = True)
df_ev["closed"]= pd.to_datetime(df_ev['closed'],infer_datetime_format = True)

In [10]:
df_ev.dtypes

id                     object
title                  object
categories             object
description            object
date           datetime64[ns]
closed         datetime64[ns]
geometries             object
dtype: object

In [11]:
df_ev.sort_values("date")

,id,title,categories,description,date,closed,geometries
4479,EONET_713,"Tupungatito Volcano, Chile-Argentina",Volcanoes,,1980-01-10,1980-01-11,"{'type': 'Point', 'coordinates': [-69.797, -33..."
4478,EONET_936,"Nyamuragira Volcano, DR Congo",Volcanoes,,1980-01-30,1980-02-24,"{'type': 'Point', 'coordinates': [29.2, -1.408]}"
4477,EONET_533,"Krakatau Volcano, Indonesia",Volcanoes,,1980-03-15,1980-12-31,"{'type': 'Point', 'coordinates': [105.423, -6...."
4476,EONET_1043,"St. Helens Volcano, Washington",Volcanoes,,1980-03-27,1986-10-28,"{'type': 'Point', 'coordinates': [-122.18, 46.2]}"
4475,EONET_886,"Lopevi Volcano, Vanuatu",Volcanoes,,1980-04-15,1980-08-20,"{'type': 'Point', 'coordinates': [168.346, -16..."
...,...,...,...,...,...,...,...
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [116.9, -16.4]}"
3,EONET_4614,"Wildfire - Constitucion, Chile",Wildfires,,2020-03-11,NaT,"{'type': 'Point', 'coordinates': [-72.42013, -..."
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [153.8, -16.9]}"
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [52.4, -14.3]}"


In [209]:
storms = df_ev[df_ev.categories== "Severe Storms"].copy()

In [210]:
storms.head()

,id,title,categories,description,date,closed,geometries
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}"


In [211]:
len(storms)

1825

In [212]:
storms["coordinates"]= storms.loc[:, ("geometries")].apply(lambda x: x["coordinates"])

In [213]:
storms

,id,title,categories,description,date,closed,geometries,coordinates
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","[-14.3, 52.4]"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","[-16.9, 153.8]"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","[-16.4, 116.9]"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","[-13.4, 118.7]"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","[-15.6, 139.5]"
...,...,...,...,...,...,...,...,...
4178,EONET_2351,Typhoon #34,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","[14.7, 131.6]"
4179,EONET_2312,Cyclone #4,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","[-10.2, 65.4]"
4180,EONET_2409,Cyclone #2,Severe Storms,,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","[-13.3, 119.4]"
4181,EONET_2410,Tropical Storm #1,Severe Storms,,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","[-9.4, 97.4]"


In [189]:
def rev(x):
    y = x[::-1]
    return y

In [190]:
storms["coordinates"] = storms.coordinates.apply(rev)

In [214]:
storms.head()

,id,title,categories,description,date,closed,geometries,coordinates
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","[-14.3, 52.4]"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","[-16.9, 153.8]"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","[-16.4, 116.9]"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","[-13.4, 118.7]"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","[-15.6, 139.5]"


In [215]:
storms["list_length"] = storms.coordinates.apply(lambda x: len(x)).copy()

In [216]:
storms = storms[~(storms.list_length < 2)]

In [217]:
storms.drop(columns = "list_length", inplace= True)

In [218]:
storms

,id,title,categories,description,date,closed,geometries,coordinates
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","[-14.3, 52.4]"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","[-16.9, 153.8]"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","[-16.4, 116.9]"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","[-13.4, 118.7]"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","[-15.6, 139.5]"
...,...,...,...,...,...,...,...,...
4178,EONET_2351,Typhoon #34,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","[14.7, 131.6]"
4179,EONET_2312,Cyclone #4,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","[-10.2, 65.4]"
4180,EONET_2409,Cyclone #2,Severe Storms,,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","[-13.3, 119.4]"
4181,EONET_2410,Tropical Storm #1,Severe Storms,,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","[-9.4, 97.4]"


In [219]:
storms["coordinates"]= storms.coordinates.apply(lambda x: (x[0], x[1])).copy()

/Users/Fabi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



As observed, we have geometries but explicit cities or countries in which the extreme events took place. For that reason we exract the latitud and longitud from ``geometries`` in order to use ``reverse_geocoder`` module:

,id,title,categories,description,date,closed,geometries,coordinates
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)"


In [222]:
storms["location"]= storms.coordinates.apply(lambda x: rg.search(x)[0]["name"])
storms["admin"]= storms.coordinates.apply(lambda x: rg.search(x)[0]["admin1"])
storms["code"]= storms.coordinates.apply(lambda x: rg.search(x)[0]["cc"])


/Users/Fabi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/Fabi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/Fabi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [224]:
storms

,id,title,categories,description,date,closed,geometries,coordinates,location,admin,code
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)",Antalaha,Sava,MG
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)",Proserpine,Queensland,AU
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)",Dampier,Western Australia,AU
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)",Alyangula,Northern Territory,AU
...,...,...,...,...,...,...,...,...,...,...,...
4178,EONET_2351,Typhoon #34,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","(14.7, 131.6)",San Policarpo,Eastern Visayas,PH
4179,EONET_2312,Cyclone #4,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","(-10.2, 65.4)",Cargados Carajos,Cargados Carajos,MU
4180,EONET_2409,Cyclone #2,Severe Storms,,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","(-13.3, 119.4)",Lailunggi,East Nusa Tenggara,ID
4181,EONET_2410,Tropical Storm #1,Severe Storms,,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","(-9.4, 97.4)",West Island,,CC


In [225]:
codes= pd.read_excel("Data_input/country_codes.xlsx")

In [226]:
codes.head()

,Country,ISO2,ISO3,Numeric
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20


In [227]:
storms= storms.merge(codes[["ISO2", "ISO3", "Country"]], 
                     left_on= "code", 
                     right_on= "ISO2", 
                     how= "left").drop(columns= "code")

In [228]:
storms

,id,title,categories,description,date,closed,geometries,coordinates,location,admin,ISO2,ISO3,Country
0,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)",Antalaha,Sava,MG,MDG,Madagascar
1,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)",Proserpine,Queensland,AU,AUS,Australia
2,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)",Dampier,Western Australia,AU,AUS,Australia
3,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
4,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)",Alyangula,Northern Territory,AU,AUS,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814,EONET_2351,Typhoon #34,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","(14.7, 131.6)",San Policarpo,Eastern Visayas,PH,PHL,Philippines (the)
1815,EONET_2312,Cyclone #4,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","(-10.2, 65.4)",Cargados Carajos,Cargados Carajos,MU,MUS,Mauritius
1816,EONET_2409,Cyclone #2,Severe Storms,,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","(-13.3, 119.4)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
1817,EONET_2410,Tropical Storm #1,Severe Storms,,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","(-9.4, 97.4)",West Island,,CC,CCK,Cocos (Keeling) Islands (the)


In [229]:
storms.drop(columns= "description", inplace= True)

In [230]:
storms.reset_index(drop= True, inplace= True)

In [231]:
storms

,id,title,categories,date,closed,geometries,coordinates,location,admin,ISO2,ISO3,Country
0,EONET_4615,Tropical Cyclone Herold,Severe Storms,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)",Antalaha,Sava,MG,MDG,Madagascar
1,EONET_4616,Tropical Cyclone Gretel,Severe Storms,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)",Proserpine,Queensland,AU,AUS,Australia
2,EONET_4612,Tropical Cyclone 21S,Severe Storms,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)",Dampier,Western Australia,AU,AUS,Australia
3,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
4,EONET_4595,Tropical Cyclone Esther,Severe Storms,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)",Alyangula,Northern Territory,AU,AUS,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...
1814,EONET_2351,Typhoon #34,Severe Storms,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","(14.7, 131.6)",San Policarpo,Eastern Visayas,PH,PHL,Philippines (the)
1815,EONET_2312,Cyclone #4,Severe Storms,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","(-10.2, 65.4)",Cargados Carajos,Cargados Carajos,MU,MUS,Mauritius
1816,EONET_2409,Cyclone #2,Severe Storms,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","(-13.3, 119.4)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
1817,EONET_2410,Tropical Storm #1,Severe Storms,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","(-9.4, 97.4)",West Island,,CC,CCK,Cocos (Keeling) Islands (the)


In [232]:
storms.to_json("storms.json")

# Data wrangling Internal displaced people

Data source: https://data.humdata.org/dataset/idmc-internally-displaced-persons-idps-new-displacement-associated-with-disasters

In [538]:
idp = pd.read_csv("Data_input/disaster_data.csv")

In [539]:
idp.head()

,ISO3,Name,Year,Start Date,Event Name,Hazard Category,Hazard Type,New Displacements
0,#country+code,#country+name,#date+year,#date+start,#description,#crisis+category,#crisis+type,#affected+idps+ind+newdisp+disaster
1,AB9,Abyei Area,2018,2018-07-01,Abyie: Flood - 01/07/2018,Weather related,Flood,2
2,AFG,Afghanistan,2008,2008-01-01,NaN,Weather related,Extreme temperature,NaN
3,AFG,Afghanistan,2008,2008-08-01,NaN,Weather related,Flood,180
4,AFG,Afghanistan,2008,2008-04-17,NaN,Geophysical,Earthquake,3250


In [540]:
idp.dtypes

ISO3                 object
Name                 object
Year                 object
Start Date           object
Event Name           object
Hazard Category      object
Hazard Type          object
New Displacements    object
dtype: object

In [541]:
idp.drop([0], inplace= True)
#Changing the data types accordingly
idp["Year"]=idp["Year"].astype(int)
idp["Start Date"] = pd.to_datetime(idp['Start Date'],infer_datetime_format = True)

In [543]:
#Changing columns names

idp.rename(columns = {"Name": "Country",
                      "Start Date": "Date",
                      "New Displacements": "Number of people affected",
                      "Event Name": "Event"},
          inplace = True)

In [544]:
idp["Hazard Category"].value_counts()

Weather related    5914
Geophysical         514
Name: Hazard Category, dtype: int64

In [545]:
idp[idp["Hazard Category"]== "Weather related"]["Hazard Type"].value_counts()

Flood                      3086
Storm                      1656
Wet mass movement           525
Wildfire                    512
Extreme temperature          95
Mass movement                23
Drought                      14
Severe winter condition       1
Wet Mass movement             1
Wet Mass Movement             1
Name: Hazard Type, dtype: int64

From the Hazard type, we are interested in ``Storm`` type:

In [546]:
idp_storms = idp[idp["Hazard Type"]== "Storm"].copy()

In [547]:
idp_storms.reset_index(drop= True, inplace= True)

In [548]:
idp_storms["Event"].value_counts()

15 countries - Hurricane Irma - 27/08/2017                                                      16
Hurricane Matthew                                                                               11
China, Guam, Northern Mariana Islands, Philippines, Thailand: Typhoon Mangkhut - 7/9/2018        7
Hurricane Earl                                                                                   6
Hurricane Irene                                                                                  6
                                                                                                ..
Indonesia: Strong Wind - Cilacap, Jepara, Kota Semarang, Wonosobo (Central Java) - 15/2/2018     1
Sri Lanka: Storm - Akurana; Kandy - 05/11/2018                                                   1
DRC: Storm - Haut-Lomami - 08/09/2017                                                            1
Gale in San Martin, Cusco, Amazonas                                                              1
Fiji : Tro

In [549]:
def get_cat(event):
    """
    Extract the category of a cyclone from its name.
    """
    if re.findall(r"Hurricane", str(event)):
        return "Hurricane"
    elif re.findall(r"Cyclone", str(event)):
        return "Cyclone"
    elif re.findall(r"Typhoon", str(event)):
        return "Typhoon"
    elif re.findall(r"Depression", str(event)):
        return "Tropical Depression"
    elif re.findall(r"Tropical Storm", str(event)):
        return "Tropical Storm"
    else:
        return "No Cyclone" 
    
    


In [550]:
idp_storms["Hazard Type"] = idp_storms["Event"].apply(get_cat)
#idp_storms.drop(columns = ["Type"], inplace= True)

In [551]:
idp_storms[idp_storms["Hazard Type"]== "Tropical Storm"]

,ISO3,Country,Year,Date,Event,Hazard Category,Hazard Type,Number of people affected
149,CHN,China,2011,2011-01-01,Tropical Storm Haima (Egay),Weather related,Tropical Storm,NaN
190,CHN,China,2016,2016-08-15,Tropical Storm Dianmu,Weather related,Tropical Storm,56000
196,CHN,China,2017,2017-08-30,China: Tropical Storm Mawar - Guangdong ; Fuji...,Weather related,Tropical Storm,78820
252,CHN,China,2018,2018-08-15,"China, DPR Korea: Tropical Storm Soulik - 24/8...",Weather related,Tropical Storm,15000
258,CHN,China,2018,2018-08-14,China: Tropical Storm Rumbia - 14/08/2018,Weather related,Tropical Storm,206900
444,DMA,Dominica,2018,2018-09-10,Dominica: Tropical Storm Isaac,Weather related,Tropical Storm,344
461,DOM,Dominican Republic,2018,2018-07-10,Dominican Republic and Puerto Rico: Tropical S...,Weather related,Tropical Storm,11740
529,GRD,Grenada,2017,2017-07-18,Grenada - St Vincent and Grenadine: Tropical S...,Weather related,Tropical Storm,140
530,GRD,Grenada,2017,2017-06-19,Grenada: Tropical Storm Bret - 19/06/2017,Weather related,Tropical Storm,6
817,KHM,Cambodia,2017,2017-07-21,"Cambodia, Laos, Thailand, Vietnam: Tropical St...",Weather related,Tropical Storm,4266


In [552]:
idp_cyclone = idp_storms[~(idp_storms["Hazard Type"]== "No Cyclone")].copy()

In [553]:
bow = ["Hurricane", "Cyclone", "Typhoon", "Storm", "Depression"]

def catch_name(name):
    """
    Extract the specific name of an event from its description
    """
    words = name.split()
    try:
        for hazard in bow:
            if hazard in words:
                return hazard + " " + words[words.index(hazard)+1]
    except:
        return ""

In [554]:
#Testing the function
catch_name("Tropical Depression Sinlaku")

'Depression Sinlaku'

In [555]:
#Creating new column with the name of the cyclone
idp_cyclone["Name"] = idp_cyclone["Event"].apply(catch_name)

In [556]:
idp_cyclone["Name"] = idp_cyclone["Name"].apply(lambda x: x if x!= "" else "Somalia Cyclone")

In [557]:
idp_cyclone.sort_values("Year", inplace= True)

In [558]:
idp_cyclone.drop(columns= ["Hazard Category"], inplace= True)

In [559]:
def get_rid(word):
    try:
        return word.replace("'", "")
    except:
        return ""

In [560]:
idp_cyclone.sort_values("Year")

,ISO3,Country,Year,Date,Event,Hazard Type,Number of people affected,Name
424,CUB,Cuba,2008,2008-08-09,Hurricane Ike,Hurricane,2616000,Hurricane Ike
1139,PHL,Philippines,2009,2009-01-01,Tropical Storm 'Isang' (Molave),Tropical Storm,7265,Storm 'Isang'
1138,PHL,Philippines,2009,2009-01-01,Typhoon Morakot (Kiko),Typhoon,34982,Typhoon Morakot
708,IND,India,2009,2009-01-01,Cyclone 'Aila',Cyclone,2300000,Cyclone 'Aila'
1284,SLV,El Salvador,2009,2009-01-01,Hurricane 'Ida',Hurricane,15000,Hurricane 'Ida'
...,...,...,...,...,...,...,...,...
38,AUS,Australia,2018,2018-03-16,Australia: Cyclone Marcus - 16/03/2018,Cyclone,130,Cyclone Marcus
1615,VUT,Vanuatu,2018,2018-03-06,Vanuatu: Tropical Cyclone Hola - 6/3/2018,Cyclone,100,Cyclone Hola
1226,PHL,Philippines,2018,2018-09-07,"China, Guam, Northern Mariana Islands, Philipp...",Typhoon,1570804,Typhoon Mangkhut
1604,VNM,Viet Nam,2018,2018-06-02,China; VietNam: Tropical Cyclone Ewiniar - 02/...,Cyclone,7,Cyclone Ewiniar


In [561]:
idp_cyclone[idp_cyclone.Name.isna()]

,ISO3,Country,Year,Date,Event,Hazard Type,Number of people affected,Name
918,MDG,Madagascar,2009,2009-01-01,Cyclones Eric and Fanele,Cyclone,4012,None
947,MEX,Mexico,2013,2013-09-12,Hurricanes Ingrid and Manuel,Hurricane,118000,None


In [562]:
idp_cyclone["Name"] = idp_cyclone["Name"].apply(get_rid)

In [563]:
idp_cyclone

,ISO3,Country,Year,Date,Event,Hazard Type,Number of people affected,Name
424,CUB,Cuba,2008,2008-08-09,Hurricane Ike,Hurricane,2616000,Hurricane Ike
1379,TWN,"Taiwan, China",2009,2009-01-01,Typhoon Morakot (Kiko),Typhoon,24950,Typhoon Morakot
1135,PHL,Philippines,2009,2009-01-01,Typhoon 'Dante' (Kujira),Typhoon,54718,Typhoon Dante
1133,PHL,Philippines,2009,2009-01-01,Typhoon 'Emong' (Chanom),Typhoon,117220,Typhoon Emong
471,FJI,Fiji,2009,2009-01-01,Cyclone Mick,Cyclone,3845,Cyclone Mick
...,...,...,...,...,...,...,...,...
1024,MNP,Northern Mariana Islands,2018,2018-10-24,"Guam, Northern Mariana Islands, Philippines: S...",Typhoon,12800,Typhoon Yutu
730,IND,India,2018,2018-10-10,India: Cyclone Titli - Odisha; Andhra Pradesh ...,Cyclone,400336,Cyclone Titli
731,IND,India,2018,2018-09-21,India: Tropical Cyclone Daye - Odisha - 21/09/...,Cyclone,3703,Cyclone Daye
461,DOM,Dominican Republic,2018,2018-07-10,Dominican Republic and Puerto Rico: Tropical S...,Tropical Storm,11740,Storm Beryl


In [574]:
idp_cyclone.to_excel("Data_output/idp_cyclone.xls")

# Further improvement of EONET - dataset

We will categorize the cyclones according with the Saffir-Simpson Scale:

* Tropical depression
* Tropical Storm
* Hurrican categories 1, 2, 3, 4, 5 

In [488]:
storms["Hazard Type"] = storms.title.apply(get_cat)

In [489]:
storms

,id,title,categories,date,closed,geometries,coordinates,location,admin,ISO2,ISO3,Country,Hazard Type
0,EONET_4615,Tropical Cyclone Herold,Severe Storms,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)",Antalaha,Sava,MG,MDG,Madagascar,Cyclone
1,EONET_4616,Tropical Cyclone Gretel,Severe Storms,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)",Proserpine,Queensland,AU,AUS,Australia,Cyclone
2,EONET_4612,Tropical Cyclone 21S,Severe Storms,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)",Dampier,Western Australia,AU,AUS,Australia,Cyclone
3,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia,Cyclone
4,EONET_4595,Tropical Cyclone Esther,Severe Storms,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)",Alyangula,Northern Territory,AU,AUS,Australia,Cyclone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814,EONET_2351,Typhoon #34,Severe Storms,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","(14.7, 131.6)",San Policarpo,Eastern Visayas,PH,PHL,Philippines (the),Typhoon
1815,EONET_2312,Cyclone #4,Severe Storms,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","(-10.2, 65.4)",Cargados Carajos,Cargados Carajos,MU,MUS,Mauritius,Cyclone
1816,EONET_2409,Cyclone #2,Severe Storms,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","(-13.3, 119.4)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia,Cyclone
1817,EONET_2410,Tropical Storm #1,Severe Storms,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","(-9.4, 97.4)",West Island,,CC,CCK,Cocos (Keeling) Islands (the),Tropical Storm


In [490]:
storms["Hazard Type"].value_counts()

Tropical Storm         844
Cyclone                375
Hurricane              300
Typhoon                293
No Cyclone               5
Tropical Depression      2
Name: Hazard Type, dtype: int64

In [567]:
storms[storms["Hazard Type"]== "No Cyclone"]

,id,title,categories,date,closed,geometries,coordinates,location,admin,ISO2,ISO3,Country,Hazard Type
35,EONET_4486,Subtropical Storm Rebekah,Severe Storms,2019-10-30,2019-11-01,"{'type': 'Point', 'coordinates': [38.3, -40.7]}","(38.3, -40.7)",Ribeira Grande,Azores,PT,PRT,Portugal,No Cyclone
82,EONET_4236,Topical Storm Alvin,Severe Storms,2019-06-25,2019-06-30,"{'type': 'Point', 'coordinates': [15.6, -106.8]}","(15.6, -106.8)",San Patricio,Jalisco,MX,MEX,Mexico,No Cyclone
84,EONET_4192,Subtropical Storm Andrea,Severe Storms,2019-05-21,2019-05-22,"{'type': 'Point', 'coordinates': [28.8, -68.7]}","(28.8, -68.7)",Hamilton,Hamilton city,BM,BMU,Bermuda,No Cyclone
127,EONET_3987,Storm Zorba,Severe Storms,2018-09-27,2018-10-01,"{'type': 'Point', 'coordinates': [31.064, 18.1...","(31.064, 18.179)",Qasr Abu Hadi,Surt,LY,LBY,Libya,No Cyclone
368,EONET_457,"Jiangsu, China Tornado June 2016",Severe Storms,2016-06-23,2016-06-30,"{'type': 'Point', 'coordinates': [33.722055248...","(33.72205524868734, 119.80453491210938)",Dongkan,Jiangsu Sheng,CN,CHN,China,No Cyclone


In [571]:
storms.sort_values("date", ascending = False, inplace= True)

In [576]:
storms.set_index("id").to_excel("Data_output/cycl_nasa.xls")